Web scrapper

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import quote

def descargar_imagenes_bing(termino_busqueda, cantidad=100, carpeta_destino="imagenes_hormigas"):
    """
    Descarga imágenes desde Bing Image Search
    
    Args:
        termino_busqueda: Término a buscar (ej: "ant close up macro")
        cantidad: Número de imágenes a descargar
        carpeta_destino: Carpeta donde guardar las imágenes
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    # URL de búsqueda de Bing Images
    url_busqueda = f"https://www.bing.com/images/search?q={quote(termino_busqueda)}&form=HDRSC2&first=1"
    
    print(f"Buscando imágenes de: {termino_busqueda}")
    
    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Buscar enlaces de imágenes
        img_tags = soup.find_all('a', class_='iusc')
        
        contador = 0
        
        for idx, img_tag in enumerate(img_tags):
            if contador >= cantidad:
                break
                
            try:
                # Extraer URL de la imagen
                m = img_tag.get('m')
                if m:
                    import json
                    img_data = json.loads(m)
                    img_url = img_data.get('murl')
                    
                    if img_url:
                        # Descargar imagen
                        print(f"Descargando imagen {contador + 1}/{cantidad}...")
                        img_response = requests.get(img_url, headers=headers, timeout=10)
                        
                        if img_response.status_code == 200:
                            # Determinar extensión
                            ext = img_url.split('.')[-1].split('?')[0][:4]
                            if ext not in ['jpg', 'jpeg', 'png', 'webp']:
                                ext = 'jpg'
                            
                            # Guardar imagen
                            filename = os.path.join(carpeta_destino, f"hormiga_{contador + 1}.{ext}")
                            with open(filename, 'wb') as f:
                                f.write(img_response.content)
                            
                            contador += 1
                            time.sleep(0.5)  # Pausa para no sobrecargar el servidor
                        
            except Exception as e:
                print(f"Error descargando imagen {idx}: {e}")
                continue
        
        print(f"\n✓ Descarga completada: {contador} imágenes guardadas en '{carpeta_destino}'")
        
    except Exception as e:
        print(f"Error en la búsqueda: {e}")

# Ejemplo de uso
if __name__ == "__main__":
    # Diferentes búsquedas para obtener variedad
    terminos = [
        "ant close up macro photography",
        "ant face macro",
        "ant portrait macro",
        "hormiga macro fotografía"
    ]
    
    # Descargar 250 imágenes de cada término (1000 total)
    for termino in terminos:
        print(f"\n{'='*50}")
        carpeta = f"hormigas_{terminos.index(termino) + 1}"
        descargar_imagenes_bing(termino, cantidad=250, carpeta_destino=carpeta)
        time.sleep(2)  # Pausa entre búsquedas


Buscando imágenes de: ant close up macro photography
Descargando imagen 1/250...
Descargando imagen 2/250...
Descargando imagen 3/250...
Descargando imagen 4/250...
Descargando imagen 5/250...
Descargando imagen 6/250...
Descargando imagen 7/250...
Descargando imagen 8/250...
Descargando imagen 9/250...
Descargando imagen 10/250...
Descargando imagen 11/250...
Descargando imagen 12/250...
Descargando imagen 13/250...
Descargando imagen 14/250...
Descargando imagen 15/250...
Descargando imagen 16/250...
Descargando imagen 17/250...
Descargando imagen 18/250...
Descargando imagen 19/250...
Descargando imagen 20/250...
Descargando imagen 21/250...
Descargando imagen 22/250...
Descargando imagen 23/250...
Descargando imagen 24/250...
Descargando imagen 25/250...
Descargando imagen 26/250...
Descargando imagen 27/250...
Descargando imagen 28/250...
Descargando imagen 29/250...
Descargando imagen 30/250...
Descargando imagen 30/250...
Descargando imagen 31/250...
Descargando imagen 32/250..

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import quote
import json

def descargar_imagenes_bing(termino_busqueda, offset, headers):
    """
    Descarga un lote de imágenes desde Bing
    """
    url = f"https://www.bing.com/images/async?q={quote(termino_busqueda)}&first={offset}&count=35&mmasync=1"
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        img_tags = soup.find_all('a', class_='iusc')
        
        urls = []
        for img_tag in img_tags:
            try:
                m = img_tag.get('m')
                if m:
                    img_data = json.loads(m)
                    img_url = img_data.get('murl')
                    if img_url:
                        urls.append(img_url)
            except:
                continue
        
        return urls
    except Exception as e:
        print(f"Error en búsqueda (offset {offset}): {e}")
        return []

def descargar_imagen(url, ruta, headers):
    """
    Descarga una imagen individual
    """
    try:
        response = requests.get(url, headers=headers, timeout=15, stream=True)
        if response.status_code == 200:
            with open(ruta, 'wb') as f:
                for chunk in response.iter_content(8192):
                    f.write(chunk)
            return True
    except:
        pass
    return False

def descargar_5k_hormigas(carpeta_destino="/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Hormigas_sucio", objetivo=5000):
    """
    Descarga 5,000 imágenes de hormigas de cuerpo completo
    """
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Términos de búsqueda variados para hormigas de cuerpo completo
    terminos = [
        "ant full body macro",
        "ant side view macro photography",
        "ant profile macro",
        "ant whole body close up",
        "ant lateral view macro",
        "ant complete body",
        "hormiga cuerpo completo macro",
        "ant specimen photography",
        "ant isolated white background",
        "ant dorsal view macro",
        "ant ventral view",
        "ant high resolution full body",
        "carpenter ant full body",
        "fire ant macro full",
        "bullet ant full body",
        "leafcutter ant complete",
        "ant species full body",
        "ant macro detailed full",
        "red ant full body macro",
        "black ant complete body"
    ]
    
    contador = 0
    indice_termino = 0
    
    print(f"🐜 Iniciando descarga de {objetivo} imágenes de hormigas...")
    print(f"📁 Carpeta destino: {carpeta_destino}\n")
    
    while contador < objetivo:
        termino_actual = terminos[indice_termino % len(terminos)]
        
        # Calcular offset - cada término exploramos hasta 300 resultados
        offset_base = (indice_termino // len(terminos)) * 300
        
        print(f"\n{'='*60}")
        print(f"🔍 Buscando: '{termino_actual}' (offset: {offset_base})")
        print(f"📊 Progreso: {contador}/{objetivo} imágenes descargadas")
        print(f"{'='*60}")
        
        # Descargar en lotes de 35 imágenes (límite de Bing por request)
        for offset in range(offset_base, offset_base + 300, 35):
            if contador >= objetivo:
                break
            
            print(f"\n🔄 Obteniendo URLs (offset {offset})...")
            urls = descargar_imagenes_bing(termino_actual, offset, headers)
            
            if not urls:
                print("⚠️  No se encontraron más imágenes en este lote")
                break
            
            print(f"✓ Encontradas {len(urls)} URLs")
            
            # Descargar cada imagen
            for url in urls:
                if contador >= objetivo:
                    break
                
                # Determinar extensión
                ext = url.split('.')[-1].split('?')[0][:4].lower()
                if ext not in ['jpg', 'jpeg', 'png', 'webp']:
                    ext = 'jpg'
                
                filename = os.path.join(carpeta_destino, f"ant_{contador + 1:05d}.{ext}")
                
                # Saltar si ya existe
                if os.path.exists(filename):
                    contador += 1
                    continue
                
                print(f"⬇️  [{contador + 1}/{objetivo}] Descargando...", end=' ')
                
                if descargar_imagen(url, filename, headers):
                    contador += 1
                    print("✓")
                else:
                    print("✗")
                
                # Pausa para no sobrecargar
                time.sleep(0.3)
            
            # Pausa entre lotes
            time.sleep(1)
        
        indice_termino += 1
        
        # Si hemos usado todos los términos y offsets, reiniciar con más variaciones
        if indice_termino >= len(terminos) * 3 and contador < objetivo:
            print("\n⚠️  Alcanzado límite de búsquedas. Reiniciando con variaciones...")
            indice_termino = 0
            time.sleep(3)
    
    print(f"\n\n{'='*60}")
    print(f"🎉 ¡DESCARGA COMPLETADA!")
    print(f"✓ {contador} imágenes guardadas en '{carpeta_destino}'")
    print(f"{'='*60}")

if __name__ == "__main__":
    descargar_5k_hormigas(carpeta_destino="hormigas_dataset", objetivo=5000)

🐜 Iniciando descarga de 5000 imágenes de hormigas...
📁 Carpeta destino: hormigas_dataset


🔍 Buscando: 'ant full body macro' (offset: 0)
📊 Progreso: 0/5000 imágenes descargadas

🔄 Obteniendo URLs (offset 0)...
✓ Encontradas 35 URLs
⬇️  [1/5000] Descargando... ✓
⬇️  [2/5000] Descargando... ✓
⬇️  [3/5000] Descargando... ✓
⬇️  [4/5000] Descargando... ✓
⬇️  [5/5000] Descargando... ✓
⬇️  [6/5000] Descargando... ✓
⬇️  [7/5000] Descargando... ✓
⬇️  [8/5000] Descargando... ✓
⬇️  [9/5000] Descargando... ✓
⬇️  [10/5000] Descargando... ✓
⬇️  [11/5000] Descargando... ✓
⬇️  [12/5000] Descargando... ✓
⬇️  [13/5000] Descargando... ✓
⬇️  [14/5000] Descargando... ✓
⬇️  [15/5000] Descargando... ✓
⬇️  [16/5000] Descargando... ✓
⬇️  [17/5000] Descargando... ✓
⬇️  [18/5000] Descargando... ✓
⬇️  [19/5000] Descargando... ✓
⬇️  [20/5000] Descargando... ✓
⬇️  [21/5000] Descargando... ✓
⬇️  [22/5000] Descargando... ✓
⬇️  [23/5000] Descargando... ✓
⬇️  [24/5000] Descargando... ✓
⬇️  [25/5000] Descargando... ✓
⬇️

In [2]:
import pandas as pd
import requests
import os
from pathlib import Path
from urllib.parse import urlparse
import time

def download_images_from_csv(csv_file, output_folder='imagenes_inaturalist'):
    """
    Descarga todas las imágenes del CSV de iNaturalist
    
    Args:
        csv_file: Ruta al archivo CSV
        output_folder: Carpeta donde se guardarán las imágenes
    """
    # Crear carpeta de salida si no existe
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Leer CSV
    print(f"Leyendo archivo CSV: {csv_file}")
    df = pd.read_csv(csv_file)
    
    # Filtrar filas que tienen URL de imagen
    df_with_images = df[df['image_url'].notna()].copy()
    total_images = len(df_with_images)
    
    print(f"Total de observaciones con imágenes: {total_images}")
    
    # Contadores
    downloaded = 0
    failed = 0
    skipped = 0
    
    # Descargar cada imagen
    for idx, row in df_with_images.iterrows():
        image_url = row['image_url']
        observation_id = row['id']
        
        # Crear nombre de archivo usando el ID de observación
        file_extension = os.path.splitext(urlparse(image_url).path)[1] or '.jpg'
        filename = f"obs_{observation_id}{file_extension}"
        filepath = os.path.join(output_folder, filename)
        
        # Saltar si ya existe
        if os.path.exists(filepath):
            print(f"[{downloaded + failed + skipped + 1}/{total_images}] Ya existe: {filename}")
            skipped += 1
            continue
        
        # Descargar imagen
        try:
            print(f"[{downloaded + failed + skipped + 1}/{total_images}] Descargando: {filename}")
            response = requests.get(image_url, timeout=30)
            response.raise_for_status()
            
            # Guardar imagen
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            downloaded += 1
            
            # Pequeña pausa para no sobrecargar el servidor
            time.sleep(0.5)
            
        except Exception as e:
            print(f"  Error descargando {filename}: {str(e)}")
            failed += 1
    
    # Resumen final
    print("\n" + "="*50)
    print("RESUMEN DE DESCARGA")
    print("="*50)
    print(f"Total de imágenes: {total_images}")
    print(f"Descargadas exitosamente: {downloaded}")
    print(f"Ya existían (omitidas): {skipped}")
    print(f"Fallidas: {failed}")
    print(f"Carpeta de destino: {os.path.abspath(output_folder)}")
    print("="*50)

if __name__ == "__main__":
    # Usar el script
    csv_file = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/observations-634556.csv/observations-634556.csv"  
    output_folder = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Mariquita"  
    
    download_images_from_csv(csv_file, output_folder)

Leyendo archivo CSV: /home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/observations-634556.csv/observations-634556.csv
Total de observaciones con imágenes: 25078
[1/25078] Descargando: obs_256898841.jpg
[2/25078] Descargando: obs_256929414.jpeg
[3/25078] Descargando: obs_256929689.jpeg
[4/25078] Descargando: obs_256930970.jpg
[5/25078] Descargando: obs_256933763.jpeg
[6/25078] Descargando: obs_256935682.jpg
[7/25078] Descargando: obs_256940587.jpg
[8/25078] Descargando: obs_256946267.jpeg
[9/25078] Descargando: obs_256946329.jpeg
[10/25078] Descargando: obs_256949555.jpg
[11/25078] Descargando: obs_256950343.jpeg
[12/25078] Descargando: obs_256951854.jpeg
[13/25078] Descargando: obs_256954219.jpg
[14/25078] Descargando: obs_256958485.jpeg
[15/25078] Descargando: obs_256963547.jpg
[16/25078] Descargando: obs_256963620.jpg
[17/25078] Descargando: obs_256965127.jpeg
[18/25078] Descargando: obs_256965614.jpg
[19/25078] Descargando: obs_256967091.jpeg
[20/25078] Descargando: obs_

KeyboardInterrupt: 

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import quote
import json

def descargar_imagenes_bing(termino_busqueda, offset, headers):
    """
    Descarga un lote de imágenes desde Bing
    """
    url = f"https://www.bing.com/images/async?q={quote(termino_busqueda)}&first={offset}&count=35&mmasync=1"
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        img_tags = soup.find_all('a', class_='iusc')
        
        urls = []
        for img_tag in img_tags:
            try:
                m = img_tag.get('m')
                if m:
                    img_data = json.loads(m)
                    img_url = img_data.get('murl')
                    if img_url:
                        urls.append(img_url)
            except:
                continue
        
        return urls
    except Exception as e:
        print(f"Error en búsqueda (offset {offset}): {e}")
        return []

def descargar_imagen(url, ruta, headers):
    """
    Descarga una imagen individual
    """
    try:
        response = requests.get(url, headers=headers, timeout=15, stream=True)
        if response.status_code == 200:
            with open(ruta, 'wb') as f:
                for chunk in response.iter_content(8192):
                    f.write(chunk)
            return True
    except:
        pass
    return False

def descargar_5k_mariquitas(carpeta_destino="mariquitas_dataset", objetivo=5000):
    """
    Descarga 5,000 imágenes de mariquitas en primer plano
    """
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Términos de búsqueda enfocados en mariquitas en primer plano
    terminos = [
        "ladybug close up macro",
        "ladybug full body macro photography",
        "ladybug isolated white background",
        "ladybug specimen photography",
        "ladybug macro detailed",
        "ladybug dorsal view macro",
        "ladybug top view close up",
        "ladybug high resolution macro",
        "ladybug red black spots macro",
        "mariquita macro primer plano",
        "coccinellidae macro photography",
        "lady beetle close up",
        "ladybird macro detailed",
        "seven spot ladybug macro",
        "asian lady beetle close up",
        "ladybug portrait macro",
        "ladybug whole body close",
        "mariquita cuerpo completo",
        "ladybug extreme close up",
        "ladybug single macro",
        "ladybug profile macro photography",
        "ladybug studio photography",
        "ladybug white background macro",
        "ladybug individual macro",
        "coccinella septempunctata macro"
    ]
    
    contador = 0
    indice_termino = 0
    
    print(f"🐞 Iniciando descarga de {objetivo} imágenes de mariquitas...")
    print(f"📁 Carpeta destino: {carpeta_destino}\n")
    
    while contador < objetivo:
        termino_actual = terminos[indice_termino % len(terminos)]
        
        # Calcular offset - cada término exploramos hasta 300 resultados
        offset_base = (indice_termino // len(terminos)) * 300
        
        print(f"\n{'='*60}")
        print(f"🔍 Buscando: '{termino_actual}' (offset: {offset_base})")
        print(f"📊 Progreso: {contador}/{objetivo} imágenes descargadas")
        print(f"{'='*60}")
        
        # Descargar en lotes de 35 imágenes (límite de Bing por request)
        for offset in range(offset_base, offset_base + 300, 35):
            if contador >= objetivo:
                break
            
            print(f"\n🔄 Obteniendo URLs (offset {offset})...")
            urls = descargar_imagenes_bing(termino_actual, offset, headers)
            
            if not urls:
                print("⚠️  No se encontraron más imágenes en este lote")
                break
            
            print(f"✓ Encontradas {len(urls)} URLs")
            
            # Descargar cada imagen
            for url in urls:
                if contador >= objetivo:
                    break
                
                # Determinar extensión
                ext = url.split('.')[-1].split('?')[0][:4].lower()
                if ext not in ['jpg', 'jpeg', 'png', 'webp']:
                    ext = 'jpg'
                
                filename = os.path.join(carpeta_destino, f"ladybug_{contador + 1:05d}.{ext}")
                
                # Saltar si ya existe
                if os.path.exists(filename):
                    contador += 1
                    continue
                
                print(f"⬇️  [{contador + 1}/{objetivo}] Descargando...", end=' ')
                
                if descargar_imagen(url, filename, headers):
                    contador += 1
                    print("✓")
                else:
                    print("✗")
                
                # Pausa para no sobrecargar
                time.sleep(0.3)
            
            # Pausa entre lotes
            time.sleep(1)
        
        indice_termino += 1
        
        # Si hemos usado todos los términos y offsets, reiniciar con más variaciones
        if indice_termino >= len(terminos) * 3 and contador < objetivo:
            print("\n⚠️  Alcanzado límite de búsquedas. Reiniciando con variaciones...")
            indice_termino = 0
            time.sleep(3)
    
    print(f"\n\n{'='*60}")
    print(f"🎉 ¡DESCARGA COMPLETADA!")
    print(f"✓ {contador} imágenes guardadas en '{carpeta_destino}'")
    print(f"{'='*60}")

if __name__ == "__main__":
    descargar_5k_mariquitas(carpeta_destino="/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Mariquita_sucio", objetivo=5000)

🐞 Iniciando descarga de 5000 imágenes de mariquitas...
📁 Carpeta destino: /home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Mariquita_sucio


🔍 Buscando: 'ladybug close up macro' (offset: 0)
📊 Progreso: 0/5000 imágenes descargadas

🔄 Obteniendo URLs (offset 0)...
✓ Encontradas 35 URLs
⬇️  [1/5000] Descargando... ✓
⬇️  [2/5000] Descargando... ✓
⬇️  [3/5000] Descargando... ✓
⬇️  [4/5000] Descargando... ✓
⬇️  [5/5000] Descargando... ✓
⬇️  [6/5000] Descargando... ✓
⬇️  [7/5000] Descargando... ✓
⬇️  [8/5000] Descargando... ✓
⬇️  [9/5000] Descargando... ✓
⬇️  [10/5000] Descargando... ✓
⬇️  [11/5000] Descargando... ✓
⬇️  [12/5000] Descargando... ✓
⬇️  [13/5000] Descargando... ✓
⬇️  [14/5000] Descargando... ✓
⬇️  [15/5000] Descargando... ✓
⬇️  [16/5000] Descargando... ✓
⬇️  [17/5000] Descargando... ✓
⬇️  [18/5000] Descargando... ✓
⬇️  [19/5000] Descargando... ✓
⬇️  [20/5000] Descargando... ✓
⬇️  [21/5000] Descargando... ✓
⬇️  [22/5000] Descargando... ✓
⬇️  [23/5000] Descargando... ✓